# NLP Syntactic Parsing exercise: Sensible PP attachment

In this exercise, we will learn about **POS tagging** and **dependency parsing** and study the well-known **PP attachment problem**.

## Introduction and POS tagging

First, let's take a look at spaCy's Part-of-Speech (POS) tagging and dependency parsing abilities. Here's how we load a sentence into a spaCy document object and view its dependency parse:

In [1]:
! python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 26.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')
test_doc = nlp('I write code.')
displacy.render(test_doc, jupyter=True,options={'compact': True})
# Note: you can add options={'compact': True} to get a more compact image

spaCy also tokenizes the sentence for you. You can view tokens and their POS tags as follows:

In [3]:
print([(token, token.pos_) for token in test_doc])

[(I, 'PRON'), (write, 'VERB'), (code, 'NOUN'), (., 'PUNCT')]


Now let's try applying this to a real dataset. NLTK includes an API for accessing many free open textual corpora, including the Project Gutenberg collection of public domain books. We'll load an array of the sentences of Jane Austen's 1811 novel *Sense and Sensibility* for our tests:

In [4]:
import nltk
nltk.download('gutenberg')
nltk.download('punkt')
from nltk.corpus import gutenberg
sentences = gutenberg.sents('austen-sense.txt')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Question 1
How many sentences are in the novel?



In [5]:
# your code here
len(sentences)

4999

There is 4,999 sentences in the novel.

## Question 2
Create a list of spaCy parsed documents from the sentences.  *Hint:* you need to reconstruct the original sentences. *Hint:* `sentences` is iterable

In [6]:
# your code here
nlps = []
for sentence in sentences:
  nlp_temp = nlp(' '.join(sentence))
  nlps.append(nlp_temp)


## Question 3
Create a flat list of tokens in all of the documents.  How many unique lowercase tokens are there?

In [7]:
# your code here
all_tokens = []
for parsed_doc in nlps:
  for token in parsed_doc :
    all_tokens.append(token)


In [8]:
import numpy as np
np.unique([token.lower_ for token in all_tokens]).shape

(6354,)

6,354 unique lowercase tokens.

## Question 4
What are the five most common lowercase verbs in the novel counting different inflections separately?

In [9]:
# your code here
all_verbs = []
for token in all_tokens:
  if token.pos_ == 'VERB':
    all_verbs.append(token.text)

In [10]:
from collections import Counter
counter = Counter(all_verbs)

In [11]:
counter.most_common(5)

[('said', 397), ('had', 246), ('know', 230), ('have', 224), ('think', 208)]

The five most common lowercase verb are : 'said','had','know','have','think'.

## Question 5
What are the five most common verbal lemmas (base forms of verbs)?

In [12]:
# your code here
all_verbs_lemma = []
for token in all_tokens:
  if token.pos_ == 'VERB':
    all_verbs_lemma.append(token.lemma_)

In [13]:
counter_lemma = Counter(all_verbs_lemma)

In [14]:
counter_lemma.most_common(5)

[('say', 608), ('have', 556), ('know', 385), ('see', 383), ('do', 355)]

The 5 most common verbal lemmas are : 'say','have','know','see','do'.

## Dependency parsing and PP attachment

As we saw above, spaCy also generates dependency parses that we can plot. These represent the grammatical relations that connect the different words and phrases in a sentence.

For the next task, we will consider how verbs and prepositional phrases can be related in sentences. (A *prepositional phrase* or *PP* is a phrase like "in the house", "on the table", "with my friend" which is headed by a prepisition like "in", "on", "with" ...).

## Question 6
What is the difference between the prepositional phrases in the sentences in (A) and those in (B)? Plot their dependency parses with `displacy.render` and look for a difference in structure.

(A)
  * I eat an apple in my room.
  * We listen to music at the theater.
  * John visited Brazil with his friend.
  
(B)
  * I see a fly in my soup.
  * She knows the man at the store.
  * I photographed a man with a hat.

**Note:** it's possible that some of the sentences above will not be parsed properly.  Use your judgement and different parsings to differentiate between the groups.

In [15]:
# your code here
sentence_1a = nlp('I eat an apple in my room.')
displacy.render(sentence_1a, jupyter=True)

In [16]:
sentence_2a = nlp('We listen to music at the theater.')
displacy.render(sentence_2a, jupyter=True)

In [17]:
sentence_3a = nlp('John visited Brazil with his friend.')
displacy.render(sentence_3a, jupyter=True)

For each of the sentences in A, the verb is linked to the PP.

In [18]:
sentence_1b = nlp('I see a fly in my soup.')
displacy.render(sentence_1b, jupyter = True)

Here, the verb is not directly linked to the PP. The verb is linked to the object which is linked to the preposition of the PP.

In [19]:
sentence_2b = nlp('She knows the man at the store.')
displacy.render(sentence_2b, jupyter = True)

In [20]:
sentence_3b = nlp('I photographed a man with a hat.')
displacy.render(sentence_3b, jupyter = True)

Here, the verb should not be linked to the PP as the PP should be linked to the object of the sentence

As you can imagine, it is not simple for the parser to decide where the prepositional phrase should be attached -- this is the **PP attachment problem**. Let's evaluate spaCy's default behavior towards PP attachment on our *Sense and Sensibility* corpus:

## Question 7
Create tuples (verb lemma, preposition lemma) for prepositional phrases attached to the verb (like (A) above). *Hint:* for a spaCy token object `token`, you can get its children with `token.children` and the child's relation to it with `child.dep_`. What are five most common (verb lemma, preposition lemma) pairs in the novel?

In [21]:
# your code here
all_verbs_prep = []
for token in all_tokens :
  if token.pos_ == 'VERB':
    for child in token.children :
      if child.dep_ == 'prep':
        all_verbs_prep.append((token.lemma_,child.lemma_))




In [22]:
counter_verb_prep = Counter(all_verbs_prep)
counter_verb_prep.most_common(5)

[(('think', 'of'), 61),
 (('talk', 'of'), 52),
 (('come', 'to'), 50),
 (('go', 'to'), 47),
 (('see', 'in'), 43)]

The five most common (verb lemma, preposition lemma) pairs in the novel are
('think','of'),('talk', 'of'),('come', 'to'),('go', 'to'),('see', 'in').

## Question 8
Do the same where the prepositional phrase is attached to the verb's object (case (B)). What are the five most common (verb lemma, preposition lemma) pairs in this case? **Hint:** what should be the verb's child's dependency type? what should be the child child's dependency type?

In [23]:
# your code here
all_verbsprep_indirect = []
for token in all_tokens :
  if token.pos_ == 'VERB':
    for child in token.children :
      if child.dep_ == 'dobj':
        for lit_child in child.children :
          if lit_child.dep_ == 'prep':
            all_verbsprep_indirect.append((token.lemma_,lit_child.lemma_))



In [24]:
counter_indirect = Counter(all_verbsprep_indirect)
counter_indirect.most_common(5)

[(('have', 'of'), 98),
 (('have', 'in'), 45),
 (('see', 'of'), 42),
 (('give', 'of'), 37),
 (('know', 'of'), 34)]

The five most common (verb lemma, preposition lemma) are
('have','of'),('have', 'in'), ('see', 'of'), ('give', 'of'), ('know', 'of')

## Bonus question
Look at a few random sentences from the corpus that are parsed as (A) or (B). Do you agree with the given parsing? Why or why not?

In [25]:
for token in nlps[2]:
  if token.pos_ == 'VERB':
    for child in token.children :
      if child.dep_ == 'prep':
        print(f'{nlps[2]} : type A')
        break
      elif child.dep == 'dobj':
        for lit_child in child:
          if lit_child.dep_ == 'prep':
            print(f'{nlps[2]} : type B')
            break

The family of Dashwood had long been settled in Sussex . : type A


It makes sense as here the verb is directly linked to the PP.

In [26]:
for token in nlps[6]:
  if token.pos_ == 'VERB':
    for child in token.children :
      if child.dep_ == 'prep':
        print(f'{nlps[6]} : type A')
        break
      elif child.dep == 'dobj':
        for lit_child in child:
          if lit_child.dep_ == 'prep':
            print(f'{nlps[6]} : type B')
            break

In the society of his nephew and niece , and their children , the old Gentleman ' s days were comfortably spent . : type A


In [27]:
displacy.render(nlps[6], jupyter = True)

It makes sense as the verb is directly linked to the PP here.